In [1]:
import spacy
import spacy_dbpedia_spotlight
import numpy as np
from bs4 import BeautifulSoup
import requests

In [4]:
nlp = spacy.load('en_core_web_lg')

nlp.add_pipe('dbpedia_spotlight', config={'dbpedia_rest_endpoint': 'http://localhost:2222/rest'})

In [6]:
text = "When Eden Hazard moved to Chelsea, he was far better than Juan Mata and also Willian.\n yeah"
doc = nlp(text)
for token in doc:
    print(token.text)

When
Eden
Hazard
moved
to
Chelsea
,
he
was
far
better
than
Juan
Mata
and
also
Willian
.

 
yeah


In [7]:
entities=[]

for ent in doc.ents:
    URL=ent._.dbpedia_raw_result['@URI']
    start=int(ent._.dbpedia_raw_result['@offset'])
    end=int(ent._.dbpedia_raw_result['@offset'])+len(ent._.dbpedia_raw_result['@surfaceForm'])
    ent_type=ent._.dbpedia_raw_result['@types'].split(",")[-1]
    entities.append([URL,start,end,ent_type])


In [8]:
print(entities)

[['http://dbpedia.org/resource/Eden_Hazard', 5, 16, 'DBpedia:SoccerPlayer'], ['http://dbpedia.org/resource/Chelsea_F.C.', 26, 33, 'DBpedia:SoccerClub'], ['http://dbpedia.org/resource/Juan_Mata', 58, 67, 'DBpedia:SoccerPlayer'], ['http://dbpedia.org/resource/Willian_(footballer,_born_1988)', 77, 84, 'DBpedia:SoccerPlayer']]


In [38]:
abstracts={}

for ent in entities:
    URL=ent[0]
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    content = soup.find('p', class_="lead").getText()
    if content not in abstracts:
        abstracts[ent[0]]=content

In [39]:
indexed=[]
prev=0
for ent in entities:
    indexed.append(text[prev:ent[1]])
    indexed.append(text[ent[1]:ent[2]])
    prev=ent[2]
indexed.append(text[prev:])
    
print(indexed)

['When ', 'Eden Hazard', ' moved to ', 'Chelsea', ', he was far better than ', 'Juan Mata', ' and also ', 'Willian', '.']
